# Implementation of a Blind Signature Scheme with RSA

- Index Number - 200193U
- Name - Chathura Gunasekara

## Added capabilities
1. Allow the use of a large prime number p and q (for example, a 32 bit number)
2. Allow for an arbitrary size file to be read from the disk as input message for generating a blind signature
3. Allow for the blind signature to be written to the disk
4. Allow for a message and a blind signature read from the disk to be verified after extracting the signature on the original message

In [103]:
# import necessary libraries
import random

## Utility Functions
The following functions are used to provide utility functions for the protocol.
1. Miller-Rabin primality test
2. Large prime number generation function
3. Prime factorization function
4. Find the generator of a group
5. Modular exponentiation function
6. Greatest common divisor function
7. Extended Euclidean algorithm function

## File Handling Functions
The following functions are used to provide file handling functions for the protocol.
1. Read file function
2. Write file function

### Miller-Rabin primality test
This function takes a number `n` and a number of iterations `k` as input and tests if the number is prime using the Miller-Rabin primality test. The function returns `True` if the number is prime and `False` if the number is composite.

### Large prime number generation function
This function takes the number of bits as its input and generates a random number and tests for primality using the Miller-Rabin primality test. If the number is prime, it returns the number, else it generates another random number and repeats the process until a prime number is found.

In [104]:
# Function to perform Miller-Rabin primality test
def miller_rabin_test(n, k=5):
    # If n is 2 or 3, it's prime
    if n == 2 or n == 3:
        return True
    # If n is less than 2 or even, it's not prime
    if n <= 1 or n % 2 == 0:
        return False
    
    # Write n-1 as d * 2^r
    r, d = 0, n - 1
    while d % 2 == 0:
        r += 1
        d //= 2
    
    # Perform k rounds of Miller-Rabin test
    for _ in range(k):
        a = random.randint(2, n - 2)
        x = pow(a, d, n)
        if x == 1 or x == n - 1:
            continue
        for _ in range(r - 1):
            x = pow(x, 2, n)
            if x == n - 1:
                break
        else:
            return False
    return True

# Function to find a large prime number using Miller-Rabin
def find_large_prime(bits):
    while True:
        # Generate a random number with the specified bit length
        p = random.getrandbits(bits)
        # Ensure p is odd
        p |= (1 << bits - 1) | 1
        # Check if p is prime using Miller-Rabin test
        if miller_rabin_test(p):
            return p

In [105]:
# Tests for the Miller-Rabin primality test (set of known tests)
def test_miller_rabin():
    # Known prime numbers
    primes = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97]
    # Known composite numbers
    composites = [1, 4, 6, 8, 9, 10, 12, 14, 15, 16, 18, 20, 21, 22, 24, 25, 26, 27, 28, 30, 32, 33, 34, 35, 36, 38, 39, 40, 42, 44, 45, 46, 48, 49, 50, 51, 52, 54, 55, 56, 57, 58, 60, 62, 63, 64, 65, 66, 68, 69, 70, 72, 74, 75, 76, 77, 78, 80, 81, 82, 84, 85, 86, 87, 88, 90, 91, 92, 93, 94, 95, 96, 98, 99, 100]
    
    # Test known prime numbers
    for p in primes:
        assert miller_rabin_test(p), f"{p} is prime"
    # Test known composite numbers
    for c in composites:
        assert not miller_rabin_test(c), f"{c} is composite"
    print("Miller Rabin - All tests passed")

# Tests for the find_large_prime function
def test_find_large_prime():
    # Test for a 32-bit prime number
    p = find_large_prime(32)
    assert miller_rabin_test(p), f"{p} is prime"
    # Test for a 64-bit prime number
    p = find_large_prime(64)
    assert miller_rabin_test(p), f"{p} is prime"
    # Test for a 128-bit prime number
    p = find_large_prime(128)
    assert miller_rabin_test(p), f"{p} is prime"
    print("Find Large prime - All tests passed")

# Run the tests
test_miller_rabin()
test_find_large_prime()


Miller Rabin - All tests passed
Find Large prime - All tests passed


### Modular exponentiation function
This function takes three numbers `g`, `s`, and `p` as input and calculates the modular exponentiation of `g^s mod p`. The algorithm efficiently calculates the modular exponentiation using **Exponentiation by Squaring**

In [106]:
# Function for modular exponentiation
def modular_exponentiation(g, s, p):
    result = 1
    
    # Update g to be within the modulus
    g = g % p 

    while s > 0:
        # If s is odd, multiply g with the result
        if s % 2 == 1:
            result = (result * g) % p

        # s must be even now
        s = s // 2
        g = (g * g) % p  # Square g
    return result


In [107]:
# Tests for the modular_exponentiation function
def test_modular_exponentiation():
    # Test for 2^3 mod 5
    assert modular_exponentiation(2, 3, 5) == 3
    # Test for 3^4 mod 7
    assert modular_exponentiation(3, 4, 7) == 4
    # Test for 5^5 mod 11
    assert modular_exponentiation(5, 5, 11) == 1
    # Test for 7^6 mod 13
    assert modular_exponentiation(7, 6, 13) == 12
    # Test for 11^7 mod 17
    assert modular_exponentiation(11, 7, 17) == 3
    print("Modular Exponentiation - All tests passed")

# Run the tests
test_modular_exponentiation()

Modular Exponentiation - All tests passed


### Greatest Common Divisor
This function takes two numbers `a` and `b` as input and returns the greatest common divisor of the two numbers using the Euclidean algorithm.

### Extended Euclidean algorithm
This function takes two numbers `a` and `b` as input and returns the greatest common divisor of the two numbers and the coefficients `x` and `y` such that `ax + by = gcd(a, b)`. 

In [108]:
# Function to get the greatest common divisor of two numbers
def gcd(a, b):
    if((a<0) or (b<0) or (a<b)):
        print("Invalid input")
        return

    while(b != 0):
        r = a % b
        a = b
        b = r
        
    return a

# Function for the extended Euclidean algorithm
def extended_gcd(a, b):
    if b == 0:
        return a, 1, 0
    else:
        d, x, y = extended_gcd(b, a % b)
        return d, y, x - y * (a // b)

In [109]:
# Tests for the gcd function
def test_gcd():
    # Test for gcd of 12 and 15
    assert gcd(15, 12) == 3
    # Test for gcd of 21 and 35
    assert gcd(35, 21) == 7
    # Test for gcd of 77 and 143
    assert gcd(143, 77) == 11
    # Test for gcd of 121 and 242
    assert gcd(242, 121) == 121
    # Test for gcd of 256 and 512
    assert gcd(512, 256) == 256
    print("GCD - All tests passed")

# Tests for the extended_gcd function
def test_extended_gcd():
    # Test for extended gcd of 12 and 15
    d, x, y = extended_gcd(15, 12)
    assert d == 3 and x == 1 and y == -1
    # Test for extended gcd of 21 and 35
    d, x, y = extended_gcd(35, 21)
    assert d == 7 and x == -1 and y == 2
    # Test for extended gcd of 77 and 143
    d, x, y = extended_gcd(143, 77)
    assert d == 11 and x == -1 and y == 2
    # Test for extended gcd of 121 and 242
    d, x, y = extended_gcd(242, 121)
    assert d == 121 and x == 0 and y == 1
    # Test for extended gcd of 256 and 512
    d, x, y = extended_gcd(512, 256)
    assert d == 256 and x == 0 and y == 1
    print("Extended GCD - All tests passed")

# Run the tests
test_gcd()
test_extended_gcd()

GCD - All tests passed
Extended GCD - All tests passed


## File Handling Functions

### Read file function
This function takes a filename as input and reads the contents of the file and returns the contents as a string.

### Write file function
This function takes a filename and data as input and writes the data to the file.

In [110]:
# Function to read a file and return its contents
def read_file(file_name):
    with open(file_name, 'r') as file:
        return file.read()
    
# Function to write data to a file
def write_file(file_name, data):
    with open(file_name, 'w') as file:
        file.write(data)

## RSA: Key pair generation

- $p, q$ - large prime numbers
- $n = pq$ - modulus
- $\phi(n) = (p-1)(q-1)$ - Euler's totient function
- $e$ - public exponent
- $d$ - private exponent

In [111]:
# Generate two large prime numbers p and q
p = find_large_prime(32)
q = find_large_prime(32)
e = 65537

# Calculate n = p * q
n = p * q
# Euler totient function of n
phi_n = (p-1)*(q-1)

# Ensure e and phi_n are coprime and e is less than phi_n
assert gcd(phi_n, e) == 1 and e < phi_n, "e and phi_n are not coprime"

# Find the modular multiplicative inverse of e mod phi_n
d = extended_gcd(e, phi_n)[1] % phi_n

print('------------ Prime Numbers ------------')
print('p:', p)
print('q:', q)
print('---------------------------------------')
print('Prime composite (n)                  :', n)
print('Euler totient function of n (phi_n)  :', phi_n)
print('---------------------------------------')
print('Secret key (d)                       :', d)
print(f'Public key (e,n)                     : {e}, {n}')

------------ Prime Numbers ------------
p: 3498937801
q: 3512679307
---------------------------------------
Prime composite (n)                  : 12290646410052783907
Euler totient function of n (phi_n)  : 12290646403041166800
---------------------------------------
Secret key (d)                       : 11987398234316361473
Public key (e,n)                     : 65537, 12290646410052783907


## Verifying Key Pair using RSA Public Key Encryption

In [112]:
# Message for signing
m = read_file('message.txt')
m = abs(hash(m))
print('Plaintext (m):', m)

# Public Key Encryption
c = modular_exponentiation(m, e, n)
print('Ciphertext (c):', c)

# Public Key Decryption
m1 = modular_exponentiation(c, d, n)
print('Decrypted message (m):', m1)

Plaintext (m): 3388189868575018481
Ciphertext (c): 773709462365132449
Decrypted message (m): 3388189868575018481


## Blind Signature Generation

In [113]:
# Generate blinding factor
k = find_large_prime(32)
bf = modular_exponentiation(k, e, n)
print('Blinding factor:', bf)

# Blinding the message
m1 = (m*bf)%n
print('Blinded message:',m1)

# Signing the blinded message
s1 = modular_exponentiation(m1, d, n)
print('Signature on blinded message:',s1)

# Save the signature to a file
write_file('signature.txt', str(s1))

Blinding factor: 8737691064108439043
Blinded message: 7243600580449947698
Signature on blinded message: 5249225632043937546


In [114]:
# Reading the signature from the file
s1 = int(read_file('signature.txt'))

# Readding the message from the file
m = read_file('message.txt')
m = abs(hash(m))

# Inverse of k
invk = extended_gcd(k, n)[1] % n 

# Unblinding the signature
s = (s1*invk) % n
print('Signature on original message:',s)

# Computing the signature directly
s0 = modular_exponentiation(m, d, n)
print('Signature on original message when directly computed:',s0)

# Verify the signature
if(s == s0):
    print('Signature verified')

Signature on original message: 5464571021981162957
Signature on original message when directly computed: 5464571021981162957
Signature verified


## Conclusion

Optimizations were done to achieve performance improvements.
1. The Miller-Rabin primality test was used to generate large prime numbers.
2. The Extended Euclidean algorithm was used to calculate the modular inverse.
3. The Exponentiation by Squaring algorithm was used to calculate the modular exponentiation.

Signed message is written to the disk and can be verified using the public key. The signed message is then read from the disk and verified using the public key.